In [1]:
import numpy as np
import pylab
import pynbody
import pynbody.plot.sph as sph
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# from analysis_mac import * 
from analysis import *

In [2]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [3]:
# path = '~/Desktop/Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
with pd.HDFStore(path) as hdf:
    keys = np.array([h[1:] for h in hdf.keys()])

# listing dwarf galaxies of simulations that are viable for study (gaseous, satellite, etc).
print(len(keys), 'Available Keys:', *keys)

19 Available Keys: h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33


### Comparing Radial Velocities ($v_{vir}$)<a class="anchor" id="velocities"></a>

We first start scaling velocities according to $v_{eff} = v/v_{vir}$. Recall that the virial -- or circular -- velocity of a halo is defined by 
$$
    v_{vir} \equiv \sqrt{ \frac{G \cdot M_{vir}}{ r_{vir} } }.
$$
Thus, we use the gravitational constant $G$ as found previously and the keys `sat_Mgas` (giving the mass contained within $R_{vir}$) and `satRvir` ($R_{vir}$ for the satellite) to compute the $v_{vir}$ for every gas particle's respective satellite.

In [4]:
#ejected, cooled, expelled, accreted = read_all_ejected_expelled()
predischarged, discharged, accreted, preheated, heated = read_discharged()

h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33
> Returning (predischarged, discharged, accreted, preheated, heated) for all available satellites <


In [5]:
# creating weight attributes for plotting.
predischarged['weight'] = predischarged['mass'] / predischarged['mass'].sum()

discharged['weight'] = discharged['mass'] / discharged['mass'].sum()

accreted['weight'] = accreted['mass'] / accreted['mass'].sum()

preheated['weight'] = preheated['mass'] / preheated['mass'].sum()

heated['weight'] = heated['mass'] / heated['mass'].sum()

In [6]:
# grav constant in units of kpc**3 Msol**-1 yr**-2.
G = 4.53943e-24


# compiling virial velocities for each particle's respective satellite.
pre_vvir = np.sqrt(G * predischarged.sat_Mgas / predischarged.satRvir)
all_vvir = np.sqrt(G * discharged.sat_Mgas / discharged.satRvir)
acc_vvir = np.sqrt(G * accreted.sat_Mgas / accreted.satRvir)
hpre_vvir = np.sqrt(G * preheated.sat_Mgas / preheated.satRvir)
hot_vvir = np.sqrt(G * heated.sat_Mgas / heated.satRvir)


# creating callable key for v/vvir ratio.
predischarged['v_per_Vvir'] = predischarged.v / pre_vvir

discharged['v_per_Vvir'] = discharged.v / all_vvir

accreted['v_per_Vvir'] = accreted.v / acc_vvir

preheated['v_per_Vvir'] = preheated.v / hpre_vvir

heated['v_per_Vvir'] = heated.v / hot_vvir

In [7]:
predischarged.sat_Mgas.min(), predischarged.satRvir.max()

(2132660.0, 64.1044930365921)

In [10]:
discharged.loc[1]

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,sat_halo,host_disk,host_halo,other_sat,IGM,angle,key,weight,Vvir,v_per_Vvir
1,4.750210,5747543,0.045106,13465.089844,26511.959760,0.000000,9.197882,0.663830,-0.093562,-4.768703,...,False,False,True,False,False,41.999436,h148_329,0.000004,2.724726e-09,1.969798e+10
1,7.443837,2402591,0.044100,12302.725586,26710.192842,7.228866,0.880580,0.029780,0.274385,-0.157013,...,True,False,False,False,False,93.677429,h229_23,0.000004,2.177714e-09,2.963705e+09
1,4.211484,458999,0.000230,48353.031250,10783.224936,3.957396,9.863266,0.576181,0.675901,4.822513,...,False,False,False,False,True,15.247141,h329_137,0.000002,2.908209e-09,1.360173e+10
1,9.383249,3947193,0.003118,16081.027344,26561.359145,0.000000,2.593600,0.091925,1.631038,-1.653249,...,True,False,False,False,False,65.050155,h329_33,0.000004,1.120630e-09,1.374876e+10


In [9]:
discharged.mass

0       28366.376150
0       29001.936444
2       27134.726367
4       26835.758536
4       26965.973341
            ...     
4348    26610.570287
4358    26755.941329
4361    26557.801505
4360    26557.970752
4368    26645.695937
Name: mass, Length: 241104, dtype: float64

In [21]:
discharged.v.min(), discharged.v_per_Vvir.max()

(0.22234978666496374, inf)

In [18]:
0.22234978666496374/9.82581935508735e-10

226291343.8865955

In [12]:
h = heated.v_per_Vvir * 10**8

heated.v_per_Vvir.max()

2.6274210290172274e-06

In [15]:
v = discharged.v_per_Vvir * 10**8


inf

In [10]:
##### plotting discharged vs. heated gas #####
bins = 100
fig, ax = plt.subplots(1,1,dpi=150, figsize=(3.5,2.5), constrained_layout=True) # change to dpi=300 for printout.


y1, x = np.histogram(heated.v_per_Vvir, bins=bins, weights=heated.weight)
bincenters = 0.5*(x[1:] + x[:-1])
ax.plot(bincenters, y1, label='Heated', c='r')

y1, x = np.histogram(discharged.v_per_Vvir, bins=bins, weights=discharged.weight)
bincenters = 0.5*(x[1:] + x[:-1])
ax.plot(bincenters, y1, label='All Discharged', c='b')


# plot horizontal line for zero velocity.
# plotrange = [0,0.33]
# ax.plot([0,0],[0,0.3],linestyle='solid',color='black',zorder=-1)


# labels, axis scaling, etc.
ax.legend(frameon=False, fontsize='8')

ax.set_title('Radial Velocities of Discharged and Hot Gas', fontsize='medium')
ax.set_xlabel(r'Virial Velocity $(kpc/yr)$')
ax.set_ylabel('Fraction Discharged Mass')
# ax.set_xlim(-50,150)
# ax.set_ylim(0,0.2)

# ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.05))
# ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
 
ax.ticklabel_format(axis="y", style="plain",scilimits=(0,0))
ax.tick_params(direction='in',which='both',right=True,top=True)
ax.grid(alpha=0.4, zorder=-1000)

# plt.savefig('plots/theta_allsats_stacked.pdf')

plt.show()

ValueError: autodetected range of [9.82581935508735e-10, inf] is not finite